In [1]:
def return_plot(estimator, plot=False):
    df['Predicted_Signal'] = estimator.predict(X)
    df.Predicted_Signal[df.Predicted_Signal > 0.5] = 1
    df.Predicted_Signal[df.Predicted_Signal < 0.5] = -1
    Cumulative_returns = np.cumsum(df[split:]['Returns'])
    df['Startegy_returns'] = df['Returns']* df['Predicted_Signal'].shift(1)
    Cumulative_Strategy_returns = np.cumsum(df[split:]['Startegy_returns'])
    print('Return: {}%'.format(round(Cumulative_Strategy_returns[-1]*100, 2)))
    
    if plot:
        plt.figure(figsize=(10,5))
        plt.plot(Cumulative_returns, color='r',label = 'Returns')
        plt.plot(Cumulative_Strategy_returns, color='g', label = 'Strategy Returns')
        plt.legend()
        plt.show()

def accuracy_plot(estimator):
    # summarize history for accuracy
    plt.plot(estimator.history.history['acc'])
    plt.plot(estimator.history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(estimator.history.history['loss'])
    plt.plot(estimator.history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

def compare_plot(model_1, model_2):
    plt.plot(model_1.history.history['val_loss'], 'r', model_2.history.history['val_loss'], 'b')
    plt.xlabel('Epochs')
    plt.ylabel('Validation Score')
    plt.legend(['first', 'second'])
    plt.show()

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import keras
from keras.models import load_model
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, Activation, TimeDistributed
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.optimizers import SGD, Adagrad, Adam
from keras.wrappers.scikit_learn import KerasClassifier

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

Using TensorFlow backend.


In [3]:
df = pd.read_csv('data/data2_M1.csv', parse_dates=['Datetime'], index_col='Datetime')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5807667 entries, 2003-08-04 00:28:00 to 2019-02-06 23:59:00
Data columns (total 10 columns):
Target                    int64
Returns                   float64
ADX_diff                  float64
CCI_diff                  float64
MOM_diff                  float64
RSI_diff                  float64
Linear_regression_diff    float64
Linear_angle_diff         float64
Linear_slope_diff         float64
Linear_intercept_diff     float64
dtypes: float64(9), int64(1)
memory usage: 487.4 MB


In [5]:
X = df.drop('Target', axis=1).values
y = df.Target.values

split = int(len(df) * 0.60)
X_train = X[:split]
X_test = X[split:]
y_train = y[:split]
y_test = y[split:]

In [6]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Final Model

In [18]:
early_stop = EarlyStopping(patience=5, monitor='val_acc')
sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
ada = Adagrad(lr=0.01, epsilon=None, decay=0.0)
adam = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

Instructions for updating:
Colocations handled automatically by placer.


In [19]:
# 7 Layers
model = Sequential()
model.add(Dense(250, activation='relu', input_shape=(X.shape[1],)))
model.add(Dropout(.5))
model.add(Dense(250, activation='relu'))
model.add(Dropout(.5))
model.add(Dense(250, activation='relu'))
model.add(Dropout(.5))
model.add(Dense(1, activation='sigmoid'))
adam = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(optimizer=adam, loss='mean_squared_error', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=30, validation_split=0.3, callbacks=[early_stop], verbose=False)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


/home/ballmdr/anaconda3/envs/tf_gpu/lib/python3.7/site-packages/keras/callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `auc_roc` which is not available. Available metrics are: val_loss,val_acc,loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


KeyboardInterrupt: 

In [35]:
return_plot(model)

/home/ballmdr/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/ballmdr/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


IndexError: -1

In [45]:
y_pred = model.predict(X_test)
y_pred = y_pred > 0.5
accuracy_plot(model)

AttributeError: 'Sequential' object has no attribute 'history'

In [46]:
print('win rate: {}%'.format(accuracy_score(y_test, y_pred)*100))

win rate: 52.93609697869239%


In [41]:


model.save('keras_53.h5')
del model

In [44]:
model = load_model('keras_53.h5')

In [7]:
def make_model(dense_layers, activation, dropout):
    '''Creates a multi-layer perceptron model
    
    dense_layers: List of layer sizes; one number per layer
    '''

    model = Sequential()
    for i, layer_size in enumerate(dense_layers, 1):
        if i == 1:
            model.add(Dense(layer_size, input_dim=X.shape[1]))
            model.add(Activation(activation))
        else:
            model.add(Dense(layer_size))
            model.add(Activation(activation))
    model.add(Dropout(dropout))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    model.compile(loss='binary_crossentropy',
                  optimizer='Adam',
                  metrics=['binary_accuracy', auc_roc])

    return model

def auc_roc(y_true, y_pred):
    # any tensorflow metric
    value, update_op = tf.metrics.auc(y_true, y_pred)

    # find all variables created for this metric
    metric_vars = [i for i in tf.local_variables() if 'auc_roc' in i.name.split('/')[1]]

    # Add metric variables to GLOBAL_VARIABLES collection.
    # They will be initialized for new session.
    for v in metric_vars:
        tf.add_to_collection(tf.GraphKeys.GLOBAL_VARIABLES, v)

    # force to update metric values
    with tf.control_dependencies([update_op]):
        value = tf.identity(value)
        return value

In [11]:
param_grid = {'dense_layers': [[32], [32, 32], [64], [64, 64], [64, 64, 32], [64, 32], [128]],
              'activation'  : ['relu', 'tanh'],
              'dropout'     : [.25, .5, .75],
              }

clf = KerasClassifier(make_model, epochs=100, batch_size=32)
grid_cv = GridSearchCV(clf, param_grid, cv=5, refit=True, return_train_score=True, n_jobs=-1, verbose=1, iid=False, error_score=np.nan, scoring='roc_auc')
early_stop = EarlyStopping(monitor='auc_roc', patience=300, verbose=1, mode='max')

grid_cv.fit(X_train, y_train, callbacks=[early_stop], verbose=2, epochs=50)
print('\nBest Score: {:.2%}'.format(grid_cv.best_score_))
print('\nBest Params:\n', pd.Series(grid_cv.best_params_))

Fitting 5 folds for each of 42 candidates, totalling 210 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
exception calling callback for <Future at 0x7f16bdfd6198 state=finished raised TerminatedWorkerError>
Traceback (most recent call last):
  File "/home/ballmdr/anaconda3/envs/tf_gpu/lib/python3.7/site-packages/sklearn/externals/joblib/externals/loky/_base.py", line 625, in _invoke_callbacks
    callback(self)
  File "/home/ballmdr/anaconda3/envs/tf_gpu/lib/python3.7/site-packages/sklearn/externals/joblib/parallel.py", line 309, in __call__
    self.parallel.dispatch_next()
  File "/home/ballmdr/anaconda3/envs/tf_gpu/lib/python3.7/site-packages/sklearn/externals/joblib/parallel.py", line 731, in dispatch_next
    if not self.dispatch_one_batch(self._original_iterator):
  File "/home/ballmdr/anaconda3/envs/tf_gpu/lib/python3.7/site-packages/sklearn/externals/joblib/parallel.py", line 759, in dispatch_one_batch
    self._dispatch(tasks)
  File "/home/ballmdr/anaconda3/envs/tf_gpu/lib/python3.7/site-packages/sklearn

TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker. The exit codes of the workers are {SIGABRT(-6), SIGABRT(-6), SIGABRT(-6)}

In [45]:
final_model = grid_cv.best_estimator_

In [46]:
return_plot(final_model)

/home/ballmdr/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/ballmdr/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


IndexError: -1

In [47]:
y_pred = final_model.predict(X_test)
print(accuracy_score(y_test, y_pred)*100)

52.872215911120946


In [51]:
model.save('h5/keras_data2_M1_53_2.h5')